In [1]:
import tensorflow as tf
import numpy as np
from random import shuffle

from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops.rnn_cell import _linear
from tensorflow.python.ops.rnn_cell import RNNCell
import math
from tensorflow.python.ops.math_ops import sigmoid
from tensorflow.python.ops.math_ops import tanh

In [2]:
class BasicRNNCell2(RNNCell):
    
    def __init__(self, num_units, input_size=None, activation=tanh):
        if input_size is not None:
            logging.warn("%s: The input_size parameter is deprecated." % self)
        self._num_units = num_units
        self._activation = activation
    
    @property
    def state_size(self):
        return self._num_units

    @property
    def output_size(self):
        return self._num_units

    def __call__(self, inputs, state, scope=None):
        """Most basic RNN: output = new_state = activation(W * input + U * state + B)."""
        with vs.variable_scope(scope or type(self).__name__):  # "BasicRNNCell"
            output = self._activation(_linear([inputs, state], self._num_units, True))
        return output, output

In [3]:
# Create a shuffled list of all binary sequnces of length N
N = 6
s = '{0:0' + str(N) + 'b}'
train_input = [s.format(i) for i in range(2**N)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti = []
for i in train_input:
    temp_list = []
    for j in i:
        temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

# A typical element of train_input at this point will be an array like
# array([[1],[0],[1],[1],[0]])

In [4]:
# Training output

train_output = []

for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*(N+1))
    temp_list[count]=1
    train_output.append(temp_list)
        
# This matches every sequence in train_input with the one-hot encoded representation
# of its classification, that is, e_i where 0 \le i \le 20 and i is the number of 1s
# that appear in the sequence

In [5]:
# Test data

NUM_EXAMPLES = 10
test_input = train_input
test_output = train_output
train_input = train_input
train_output = train_output

# TODO at the moment we use the same test and training data

In [6]:
# Model

data = tf.placeholder(tf.float32, [None,N,1])
target = tf.placeholder(tf.float32, [None,N+1])

num_hidden = 5
#cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)
#cell = tf.nn.rnn_cell.BasicRNNCell(num_hidden)
cell = BasicRNNCell2(num_hidden)
val, state = tf.nn.dynamic_rnn(cell,data,dtype=tf.float32)
val = tf.transpose(val,[1,0,2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

weight = tf.Variable(tf.truncated_normal([num_hidden,N+1]))
bias = tf.Variable(tf.constant(0.1, shape=[N+1]))

prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
cross_entropy = -tf.reduce_sum(target * tf.log(prediction))

optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))
                                                        

//anaconda/lib/python2.7/site-packages/tensorflow/python/ops/gradients.py:89: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [7]:
# len(train_input)
print(len(test_input))
print(len(test_output))
print(train_output)

64
64
[[0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0], [0, 

In [8]:
# Run

init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

sess.run(minimize,{data: train_input, target: train_output})
incorrect = sess.run(error,{data: test_input, target: test_output})
print('error {:3.1f}%'.format(100 * incorrect))
sess.close()
        
# TODO: at the moment there are no batches

error 82.8%
